<a href="https://colab.research.google.com/github/nirabhromakhal/Resnet50-Brain-Dead/blob/main/Resnet50_Brain_Dead.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow

In [3]:
!unzip /content/drive/MyDrive/brain_dead_emotion_detection.zip

Archive:  /content/drive/MyDrive/brain_dead_emotion_detection.zip
   creating: brain_dead_emotion_detection/Test/
   creating: brain_dead_emotion_detection/Test/angry/
  inflating: brain_dead_emotion_detection/Test/angry/cartoon_52_1229.png  
  inflating: brain_dead_emotion_detection/Test/angry/cartoon_52_681.png  
  inflating: brain_dead_emotion_detection/Test/angry/cartoon_52_7161.png  
  inflating: brain_dead_emotion_detection/Test/angry/cartoon_52_7605.png  
  inflating: brain_dead_emotion_detection/Test/angry/cartoon_52_8778.png  
  inflating: brain_dead_emotion_detection/Test/angry/cartoon_52_896.png  
  inflating: brain_dead_emotion_detection/Test/angry/cartoon_52_9606.png  
  inflating: brain_dead_emotion_detection/Test/angry/cartoon_52_9935.png  
  inflating: brain_dead_emotion_detection/Test/angry/cartoon_53_4584.png  
  inflating: brain_dead_emotion_detection/Test/angry/cartoon_53_5218.png  
  inflating: brain_dead_emotion_detection/Test/angry/cartoon_53_548.png  
  inflatin

In [5]:
import os
import numpy as np
import pandas as pd
import shutil

def prepare_df(data_type):
    X = []
    y = []

    angry = '/content/brain_dead_emotion_detection/' + data_type + '/angry/'
    sad = '/content/brain_dead_emotion_detection/' + data_type + '/sad/'
    happy = '/content/brain_dead_emotion_detection/' + data_type + '/happy/'
    neutral = '/content/brain_dead_emotion_detection/' + data_type + '/neutral/'

    labels = ['angry', 'sad', 'happy', 'neutral']

    dest_dir = "/content/drive/MyDrive/Resnet50/Brain Dead/" + data_type + "ing/"

    paths = [angry, sad, happy, neutral]
    for label in range(4):
      for file in os.listdir(paths[label]):
          # Image
          X.append(file)
          # Label
          y.append(labels[label])
          # Copy file to different dir (comment this line after running this cell once)
          #shutil.copy(paths[label]+file, dest_dir)


    X = np.array(X)
    y = np.array(y)

    df = pd.DataFrame()
    df['filename'] = X
    df['label'] = y

    return df


df_train = prepare_df('Train')
df_test = prepare_df('Test')
df_train.head()

,filename,label
0,cartoon_45_4392.png,angry
1,cartoon_37_3184.png,angry
2,cartoon_17_6551.png,angry
3,cartoon_13_9865.png,angry
4,cartoon_19_3436.png,angry


In [6]:
from keras.preprocessing.image import ImageDataGenerator

# Create the ImageDataGenerator object
train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

val_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
) 

# Generate batches and augment the images
train_generator = train_datagen.flow_from_dataframe(
    df_train,
    directory='/content/drive/MyDrive/Resnet50/Brain Dead/Training/',
    x_col='filename',
    y_col='label',
    class_mode='categorical',
    target_size=(64, 64),
)

val_generator = train_datagen.flow_from_dataframe(
    df_test,
    directory='/content/drive/MyDrive/Resnet50/Brain Dead/Testing/',
    x_col='filename',
    y_col='label',
    class_mode='categorical',
    target_size=(64, 64),
)

Found 1810 validated image filenames belonging to 4 classes.
Found 369 validated image filenames belonging to 4 classes.


In [ ]:
# model 1 
import keras
from keras.applications import ResNet50

# Initialize the Pretrained Model
feature_extractor = ResNet50(weights='imagenet', 
                             input_shape=(64, 64, 3),
                             include_top=False)

# Set this parameter to make sure it's not being trained
feature_extractor.trainable = False

# Set the input layer
input_ = keras.Input(shape=(64, 64, 3))

# Set the feature extractor layer
x = feature_extractor(input_, training=False)

# Set the end layers
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128,kernel_initializer='he_uniform')(x)
x = keras.layers.Dropout(0.5)(x)

# Set the final layer with softmax activation function
output_ = keras.layers.Dense(4, activation='softmax')(x)

# Create the new model object
model = keras.Model(input_, output_)

# Compile it
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

# Print The Summary of The Model
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 182, 182, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 6, 6, 2048)        23587712  
                                                                 
 global_average_pooling2d_2   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_4 (Dense)             (None, 4)                 8196      
                                                                 
Total params: 23,595,908
Trainable params: 8,196
Non-trainable params: 23,587,712
_________________________________________________________________


In [9]:
# model 2
import keras
from keras.applications import ResNet50, VGG16

# Initialize the Pretrained Model
feature_extractor = ResNet50(weights='imagenet', 
                             input_shape=(64, 64, 3),
                             include_top=False)

# Set this parameter to make sure it's not being trained
feature_extractor.trainable = False

# Set the input layer
input_ = keras.Input(shape=(64, 64, 3))

# Set the feature extractor layer
x = feature_extractor(input_, training=False)

# Set the end layers
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(64,kernel_initializer='he_uniform')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)

# Set the final layer with softmax activation function
output_ = keras.layers.Dense(4, activation='softmax')(x)

# Create the new model object
model = keras.Model(input_, output_)

# Compile it
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

# Print The Summary of The Model
model.summary()

94765736/94765736 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 resnet50 (Functional)       (None, 2, 2, 2048)        23587712  
                                                                 
 dropout (Dropout)           (None, 2, 2, 2048)        0         
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 batch_normalization (BatchN  (None, 8192)             32768     
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 64)                524352 

In [ ]:
model.fit(train_generator, epochs=20, validation_data=val_generator)

/usr/local/lib/python3.9/dist-packages/keras/preprocessing/image.py:1861: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/keras/preprocessing/image.py:1871: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


Epoch 1/20
57/57 [==============================] - 370s 6s/step - loss: 0.6278 - accuracy: 0.3663 - val_loss: 0.5676 - val_accuracy: 0.4363
Epoch 2/20
57/57 [==============================] - 9s 164ms/step - loss: 0.5082 - accuracy: 0.4829 - val_loss: 0.5098 - val_accuracy: 0.5068
Epoch 3/20
57/57 [==============================] - ETA: 0s - loss: 0.4815 - accuracy: 0.5149

In [8]:
import keras
model = keras.models.load_model('/content/drive/MyDrive/Resnet50/Brain Dead/model')

OSError: ignored

In [ ]:
model.save('/content/drive/MyDrive/Resnet50/Brain Dead/model')